<a href="https://colab.research.google.com/github/Devacodes-dev/Machine-learning-journey/blob/main/Community_powered_music_discovery_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q implicit
!pip install -q kaggle scikit-learn umap-learn pandas tqdm joblib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install -q implicit
try:
    import implicit
    print("✅ implicit library is installed successfully.")
except ModuleNotFoundError:
    print("❌ implicit library is not installed.")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
✅ implicit library is installed successfully.


/usr/local/lib/python3.12/dist-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: CUDA driver version is insufficient for CUDA runtime version (/tmp/pip-install-pduv2qt4/implicit_40f9bf7ba88e47618937af04d57d0855/./implicit/gpu/utils.h:71)'
  warnings.warn(


In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import joblib


In [ ]:
from google.colab import files
files.upload()  # Upload your kaggle.json

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


Saving kaggle.json to kaggle.json


In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets list -s spotify

ref                                                       title                                    size  lastUpdated                 downloadCount  voteCount  usabilityRating  
--------------------------------------------------------  ---------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
dhruvildave/spotify-charts                                Spotify Charts                      991225045  2022-02-09 19:49:08.213000          22077        351  1.0              
salvatorerastelli/spotify-and-youtube                     Spotify and Youtube                   9386454  2023-03-20 15:43:25.910000          34661        625  1.0              
bricevergnou/spotify-recommendation                       Spotify Recommendation                  30201  2021-07-27 20:06:27.080000          19614        188  1.0              
andrewmvd/spotify-playlists                               Spotify Playlists                   191756954  2021-11-15

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("undefinenull/million-song-dataset-spotify-lastfm")

print("Path to dataset files:", path)

100%|██████████| 639M/639M [00:16<00:00, 39.7MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/undefinenull/million-song-dataset-spotify-lastfm/versions/1


In [ ]:
!ls /content


drive  sample_data


In [ ]:
!kaggle datasets list -s "music tracks artists interactions"

ref                                                            title                                                     size  lastUpdated                 downloadCount  voteCount  usabilityRating  
-------------------------------------------------------------  --------------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
yukawithdata/spotify-top-tracks-2023                           Spotify Top 50 Tracks 2023                                3805  2024-02-07 05:52:12.670000           3367         46  1.0              
yukawithdata/beyonce-track-attribute-data                       Spotify - Beyoncé's Track Data                          23582  2024-03-13 21:25:51.133000           1130         27  1.0              
yukawithdata/taylor-swift-the-eras-tour-official-setlist-data  Taylor Swift | The Eras Tour Official Setlist Data        2619  2024-05-12 00:14:06.090000           1119         25  1.0              
thede

In [ ]:
music_info_df = pd.read_csv('/root/.cache/kagglehub/datasets/undefinenull/million-song-dataset-spotify-lastfm/versions/1/Music Info.csv')
listening_history_df = pd.read_csv('/root/.cache/kagglehub/datasets/undefinenull/million-song-dataset-spotify-lastfm/versions/1/User Listening History.csv')

print("Music Info DataFrame head:")
display(music_info_df.head())

print("\nUser Listening History DataFrame head:")
display(listening_history_df.head())

Music Info DataFrame head:


,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,TRIOREW128F424EAF0,Mr. Brightside,The Killers,https://p.scdn.co/mp3-preview/4d26180e6961fd46...,09ZQ5TmUG8TSL56n0knqrj,"rock, alternative, indie, alternative_rock, in...",NaN,2004,222200,0.355,...,1,-4.360,1,0.0746,0.001190,0.000000,0.0971,0.240,148.114,4
1,TRRIVDJ128F429B0E8,Wonderwall,Oasis,https://p.scdn.co/mp3-preview/d012e536916c927b...,06UfBBDISthj1ZJAtX4xjj,"rock, alternative, indie, pop, alternative_roc...",NaN,2006,258613,0.409,...,2,-4.373,1,0.0336,0.000807,0.000000,0.2070,0.651,174.426,4
2,TROUVHL128F426C441,Come as You Are,Nirvana,https://p.scdn.co/mp3-preview/a1c11bb1cb231031...,0keNu0t0tqsWtExGM3nT1D,"rock, alternative, alternative_rock, 90s, grunge",RnB,1991,218920,0.508,...,4,-5.783,0,0.0400,0.000175,0.000459,0.0878,0.543,120.012,4
3,TRUEIND128F93038C4,Take Me Out,Franz Ferdinand,https://p.scdn.co/mp3-preview/399c401370438be4...,0ancVQ9wEcHVd0RrGICTE4,"rock, alternative, indie, alternative_rock, in...",NaN,2004,237026,0.279,...,9,-8.851,1,0.0371,0.000389,0.000655,0.1330,0.490,104.560,4
4,TRLNZBD128F935E4D8,Creep,Radiohead,https://p.scdn.co/mp3-preview/e7eb60e9466bc3a2...,01QoK9DA7VTeTSE3MNzp4I,"rock, alternative, indie, alternative_rock, in...",RnB,2008,238640,0.515,...,7,-9.935,1,0.0369,0.010200,0.000141,0.1290,0.104,91.841,4



User Listening History DataFrame head:


,track_id,user_id,playcount
0,TRIRLYL128F42539D1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1
1,TRFUPBA128F934F7E1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1
2,TRLQPQJ128F42AA94F,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1
3,TRTUCUY128F92E1D24,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1
4,TRHDDQG12903CB53EE,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1


In [ ]:
!ls /kaggle/input/million-song-dataset-spotify-lastfm

ls: cannot access '/kaggle/input/million-song-dataset-spotify-lastfm': No such file or directory


In [ ]:
!ls /content


drive  sample_data


In [ ]:
music_info_df = pd.read_csv('/root/.cache/kagglehub/datasets/undefinenull/million-song-dataset-spotify-lastfm/versions/1/Music Info.csv')
listening_history_df = pd.read_csv('/root/.cache/kagglehub/datasets/undefinenull/million-song-dataset-spotify-lastfm/versions/1/User Listening History.csv')

print("Music Info shape:", music_info_df.shape)
print("User-song interactions shape:", listening_history_df.shape)
music_info_df.head()

Music Info shape: (50683, 21)
User-song interactions shape: (9711301, 3)


,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,TRIOREW128F424EAF0,Mr. Brightside,The Killers,https://p.scdn.co/mp3-preview/4d26180e6961fd46...,09ZQ5TmUG8TSL56n0knqrj,"rock, alternative, indie, alternative_rock, in...",NaN,2004,222200,0.355,...,1,-4.360,1,0.0746,0.001190,0.000000,0.0971,0.240,148.114,4
1,TRRIVDJ128F429B0E8,Wonderwall,Oasis,https://p.scdn.co/mp3-preview/d012e536916c927b...,06UfBBDISthj1ZJAtX4xjj,"rock, alternative, indie, pop, alternative_roc...",NaN,2006,258613,0.409,...,2,-4.373,1,0.0336,0.000807,0.000000,0.2070,0.651,174.426,4
2,TROUVHL128F426C441,Come as You Are,Nirvana,https://p.scdn.co/mp3-preview/a1c11bb1cb231031...,0keNu0t0tqsWtExGM3nT1D,"rock, alternative, alternative_rock, 90s, grunge",RnB,1991,218920,0.508,...,4,-5.783,0,0.0400,0.000175,0.000459,0.0878,0.543,120.012,4
3,TRUEIND128F93038C4,Take Me Out,Franz Ferdinand,https://p.scdn.co/mp3-preview/399c401370438be4...,0ancVQ9wEcHVd0RrGICTE4,"rock, alternative, indie, alternative_rock, in...",NaN,2004,237026,0.279,...,9,-8.851,1,0.0371,0.000389,0.000655,0.1330,0.490,104.560,4
4,TRLNZBD128F935E4D8,Creep,Radiohead,https://p.scdn.co/mp3-preview/e7eb60e9466bc3a2...,01QoK9DA7VTeTSE3MNzp4I,"rock, alternative, indie, alternative_rock, in...",RnB,2008,238640,0.515,...,7,-9.935,1,0.0369,0.010200,0.000141,0.1290,0.104,91.841,4


In [ ]:
audio_feats = ['danceability','energy','loudness','speechiness',
               'acousticness','instrumentalness','liveness','valence','tempo']

tracks = music_info_df.dropna(subset=audio_feats + ['genre','track_id'])

In [ ]:
# Scale audio features
scaler = StandardScaler()
X_audio = scaler.fit_transform(tracks[audio_feats])

# Encode genre
tfidf = TfidfVectorizer(max_features=200)
genre_vec = tfidf.fit_transform(tracks['genre'].astype(str))

# Reduce genre dimensions
svd = TruncatedSVD(n_components=10, random_state=42)
genre_emb = svd.fit_transform(genre_vec)

# Combine audio + genre embeddings
track_content_emb = np.hstack([X_audio, genre_emb])
print("Content embedding shape:", track_content_emb.shape)

Content embedding shape: (22348, 19)


In [ ]:
from scipy.sparse import csr_matrix

user_ids = listening_history_df['user_id'].unique()
track_ids = tracks['track_id'].unique()

user2idx = {u:i for i,u in enumerate(user_ids)}
track2idx = {t:i for i,t in enumerate(track_ids)}

usersongs = listening_history_df[listening_history_df['track_id'].isin(track2idx)]
rows = usersongs['user_id'].map(user2idx)
cols = usersongs['track_id'].map(track2idx)
vals = np.ones(len(rows))  # implicit feedback

interaction = csr_matrix((vals, (rows, cols)), shape=(len(user2idx), len(track2idx)))
print("Interaction matrix shape:", interaction.shape)

Interaction matrix shape: (962037, 22348)


In [ ]:
# Use implicit ALS
import implicit
model_cf = implicit.als.AlternatingLeastSquares(factors=50, regularization=0.1, iterations=15)
model_cf.fit(interaction.T * 15)  # scale by confidence

print("✅ ALS model trained")

/usr/local/lib/python3.12/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
/usr/local/lib/python3.12/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.17667555809020996 seconds
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

✅ ALS model trained


In [ ]:
# Use implicit ALS
import implicit
model_cf = implicit.als.AlternatingLeastSquares(factors=50, regularization=0.1, iterations=15)
model_cf.fit(interaction.T * 15)  # scale by confidence

print("✅ ALS model trained")

/usr/local/lib/python3.12/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.14211130142211914 seconds
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

✅ ALS model trained


In [ ]:
def recommend_hybrid(user_id, top_k=10, content_weight=0.3):
    """
    Hybrid recommender: ALS + content embeddings
    """
    if user_id not in user2idx:
        # Cold-start: most popular tracks
        # Assuming 'popularity' is a column in tracks DataFrame
        if 'popularity' in tracks.columns:
             return tracks.sort_values('popularity', ascending=False).head(top_k)[
                 ['name','artist','genre','popularity']
             ]
        else:
             print("Warning: 'popularity' column not found. Cannot provide cold-start recommendations based on popularity.")
             return tracks.head(top_k)[['name','artist','genre']] # Return first k tracks as fallback


    uidx = user2idx[user_id]

    # ALS collaborative filtering scores
    scores_cf = model_cf.user_factors[uidx] @ model_cf.item_factors.T
    top_items = np.argsort(-scores_cf)[:top_k*5]

    rec_df = tracks.iloc[top_items].copy()
    rec_df['cf_score'] = scores_cf[top_items]

    # Content similarity
    user_items = interaction[uidx].indices
    if len(user_items) > 0:
        user_profile = track_content_emb[user_items].mean(axis=0).reshape(1,-1)
        sim = cosine_similarity(user_profile, track_content_emb[top_items])
        rec_df['content_sim'] = sim.flatten()
        rec_df['score'] = (1-content_weight)*rec_df['cf_score'] + content_weight*rec_df['content_sim']
    else:
        rec_df['score'] = rec_df['cf_score']

    return rec_df.sort_values('score', ascending=False).head(top_k)[
        ['name','artist','genre','score']
    ]

In [ ]:
example_user = list(user2idx.keys())[6]

def recommend_hybrid(user_input, top_k=10, content_weight=0.3):
    """
    Hybrid recommender: ALS + content embeddings
    Handles both existing user_id (string) and a list of track_ids (for cold-start)
    """
    if isinstance(user_input, str):
        user_id = user_input
        if user_id not in user2idx:
            # Cold-start for new user_id string: most popular tracks
            if 'popularity' in tracks.columns:
                 return tracks.sort_values('popularity', ascending=False).head(top_k)[
                     ['name','artist','genre','popularity']
                 ]
            else:
                 print("Warning: 'popularity' column not found. Cannot provide cold-start recommendations based on popularity.")
                 return tracks.head(top_k)[['name','artist','genre']] # Return first k tracks as fallback

        uidx = user2idx[user_id]

        # ALS collaborative filtering scores
        scores_cf = model_cf.user_factors[uidx] @ model_cf.item_factors.T
        top_items_indices = np.argsort(-scores_cf)[:top_k*5]

        # Get the actual track_ids from the indices, ensuring indices are valid
        valid_top_items_indices = [i for i in top_items_indices if i < len(track2idx.keys())]
        recommended_track_ids = [list(track2idx.keys())[i] for i in valid_top_items_indices]

        rec_df = tracks[tracks['track_id'].isin(recommended_track_ids)].copy()
        # Ensure the order of recommendations matches the scores
        rec_df = rec_df.set_index('track_id').loc[recommended_track_ids].reset_index()

        rec_df['cf_score'] = scores_cf[valid_top_items_indices]

        # Content similarity for existing users based on their history
        user_items = interaction[uidx].indices
        if len(user_items) > 0:
            user_profile = track_content_emb[user_items].mean(axis=0).reshape(1,-1)
            recommended_track_content_emb = track_content_emb[valid_top_items_indices]
            sim = cosine_similarity(user_profile, recommended_track_content_emb)
            rec_df['content_sim'] = sim.flatten()
            rec_df['score'] = (1-content_weight)*rec_df['cf_score'] + content_weight*rec_df['content_sim']
        else:
            rec_df['score'] = rec_df['cf_score'] # Fallback to CF score if no history


    elif isinstance(user_input, list):
        # Cold-start for list of track_ids: content-based similarity
        input_track_ids = user_input
        # Ensure all input track_ids exist in our tracks data
        valid_input_track_ids = [tid for tid in input_track_ids if tid in tracks['track_id'].values]

        if not valid_input_track_ids:
            print("Warning: None of the provided track IDs are in the dataset.")
            return tracks.head(top_k)[['name','artist','genre']] # Return first k tracks as fallback

        # Get the positional indices of the valid input track IDs within the 'tracks' DataFrame
        input_track_indices_in_tracks = tracks.index.get_indexer(tracks[tracks['track_id'].isin(valid_input_track_ids)].index)
        input_track_content_emb = track_content_emb[input_track_indices_in_tracks]

        # Create a temporary user profile from the average of input track embeddings
        user_profile = input_track_content_emb.mean(axis=0).reshape(1,-1)

        # Calculate content similarity with all tracks
        all_track_content_emb = track_content_emb
        sim_scores = cosine_similarity(user_profile, all_track_content_emb).flatten()

        # Get top items based on content similarity, excluding the input tracks
        # Create a boolean mask to exclude input tracks
        mask = ~tracks['track_id'].isin(valid_input_track_ids)
        filtered_sim_scores = sim_scores[mask]
        filtered_tracks = tracks[mask].copy()

        top_items_indices_filtered = np.argsort(-filtered_sim_scores)[:top_k]

        recommended_track_ids = filtered_tracks.iloc[top_items_indices_filtered]['track_id'].tolist()

        rec_df = tracks[tracks['track_id'].isin(recommended_track_ids)].copy()
        rec_df = rec_df.set_index('track_id').loc[recommended_track_ids].reset_index()

        # Assign content similarity score
        rec_df['score'] = filtered_sim_scores[top_items_indices_filtered]

    else:
        raise TypeError("user_input must be a user_id (string) or a list of track_ids.")

    return rec_df.sort_values('score', ascending=False).head(top_k)[
        ['name','artist','genre','score']
    ]

recommend_hybrid(example_user, top_k=10)

,name,artist,genre,score
31,Lucky,Radiohead,Rock,0.240107
24,How You Remind Me,Nickelback,Rock,0.107218
9,Use Somebody,Kings of Leon,Electronic,0.092983
27,Fortunate Son,Creedence Clearwater Revival,Rock,0.079598
16,When the Sun Goes Down,Arctic Monkeys,Rock,0.078041
19,Violet Hill,Coldplay,Rock,0.067855
1,Hysteria,Muse,Rock,0.060683
23,Light My Fire,The Doors,Rock,0.058601
0,In Bloom,Nirvana,Punk,0.058078
2,Smile Like You Mean It,The Killers,Rock,0.054299


In [ ]:
joblib.dump({
      'model_cf': model_cf,
          'scaler': scaler,
              'tfidf': tfidf,
                  'svd': svd,
                      'track_content_emb': track_content_emb,
                          'user2idx': user2idx,
                              'track2idx': track2idx,
                                  'tracks_meta': tracks
                                  }, '/content/music_recommender_hybrid.pkl')

print("✅ Saved hybrid recommender as /content/music_recommender_hybrid.pkl")

✅ Saved hybrid recommender as /content/music_recommender_hybrid.pkl


In [ ]:
# Pick a random user_id from your dataset
example_user = list(user2idx.keys())[1]  # first user
recommendations = recommend_hybrid(example_user, top_k=10)  # or recommend_for_user_content()
recommendations


,name,artist,genre,score
11,Feel Good Inc.,Gorillaz,RnB,0.208411
22,We Will Rock You,Queen,Rock,0.164508
6,Losing My Religion,R.E.M.,Pop,0.105353
28,Banquet,Bloc Party,Rock,0.092016
17,Hey There Delilah,Plain White T's,Rock,0.088707
30,Fluorescent Adolescent,Arctic Monkeys,Pop,0.075049
26,1979,The Smashing Pumpkins,Rock,0.064749
3,Heart-Shaped Box,Nirvana,Rock,0.042740
29,Buddy Holly,Weezer,Rock,0.024902
13,The Scientist,Coldplay,Rock,0.004488


In [ ]:
recommendations = recommend_hybrid("new_user_123", top_k=10)
recommendations


,name,artist,genre
2,Come as You Are,Nirvana,RnB
4,Creep,Radiohead,RnB
8,The Scientist,Coldplay,Rock
10,Under the Bridge,Red Hot Chili Peppers,Pop
12,Feel Good Inc.,Gorillaz,RnB
13,Chop Suey!,System of a Down,Metal
23,Use Somebody,Kings of Leon,Electronic
25,Paranoid Android,Radiohead,Jazz
32,Stairway to Heaven,Led Zeppelin,Punk
39,Losing My Religion,R.E.M.,Pop


In [ ]:
# User listened to 3 random tracks
example_history = list(tracks['track_id'].sample(10))
recommendations = recommend_hybrid(example_history, top_k=10)
recommendations

,name,artist,genre,score
0,Dinner at Deviant's Palace,Cradle of Filth,Metal,0.845164
1,Demontain,Ufomammut,Metal,0.818828
2,With Portfolio,Mogwai,Rock,0.797388
3,Deathzone,Apocalyptica,Metal,0.762877
4,A Saucerful of Secrets,Pink Floyd,Rock,0.761271
5,So Here We Are (Four Tet Remix),Bloc Party,Rock,0.755231
6,"To Hold You, Drum",Liars,Rock,0.744836
7,The Lighthouse,Interpol,Rock,0.741994
8,The Things Apart,Dagoba,Metal,0.736885
9,Acronym Love,Riverside,Rock,0.735353
